## NDVI calculation based on snappy examples

In [1]:
import numpy
from snappy import Product, ProductData, ProductIO, ProductUtils, FlagCoding

file = '/eodata/Envisat/Meris/FRS/2012/04/08/MER_FRS_1PPEPA20120408_105857_000005063113_00267_52867_0978.N1'

print("Reading...")
product = ProductIO.readProduct(file)
width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()

print("Product:     %s, %s" % (name, description))
print("Raster size: %d x %d pixels" % (width, height))
print("Start time:  " + str(product.getStartTime()))
print("End time:    " + str(product.getEndTime()))
print("Bands:       %s" % (list(band_names)))


b7 = product.getBand('radiance_7')
b10 = product.getBand('radiance_10')
ndviProduct = Product('NDVI', 'NDVI', width, height)
ndviBand = ndviProduct.addBand('ndvi', ProductData.TYPE_FLOAT32)
ndviFlagsBand = ndviProduct.addBand('ndvi_flags', ProductData.TYPE_UINT8)
writer = ProductIO.getProductWriter('BEAM-DIMAP')

ProductUtils.copyGeoCoding(product, ndviProduct)

ndviFlagCoding = FlagCoding('ndvi_flags')
ndviFlagCoding.addFlag("NDVI_LOW", 1, "NDVI below 0")
ndviFlagCoding.addFlag("NDVI_HIGH", 2, "NDVI above 1")
group = ndviProduct.getFlagCodingGroup()
#print(dir(group))
group.add(ndviFlagCoding)

ndviFlagsBand.setSampleCoding(ndviFlagCoding)

ndviProduct.setProductWriter(writer)
ndviProduct.writeHeader('snappy_ndvi_output.dim')

r7 = numpy.zeros(width, dtype=numpy.float32)
r10 = numpy.zeros(width, dtype=numpy.float32)

print("Writing...")

for y in range(height):
    print("processing line ", y, " of ", height)
    r7 = b7.readPixels(0, y, width, 1, r7)
    r10 = b10.readPixels(0, y, width, 1, r10)

    ndvi = (r10 - r7) / (r10 + r7)
    ndviBand.writePixels(0, y, width, 1, ndvi)
    ndviLow = ndvi < 0.0
    ndviHigh = ndvi > 1.0
    ndviFlags = numpy.array(ndviLow + 2 * ndviHigh, dtype=numpy.int32)
    ndviFlagsBand.writePixels(0, y, width, 1, ndviFlags)

ndviProduct.closeIO()

print("Done.")

Reading...
Product:     MER_FRS_1PPEPA20120408_105857_000005063113_00267_52867_0978.N1, MERIS Full Resolution Geolocated and Calibrated TOA Radiance
Raster size: 4481 x 11521 pixels
Start time:  08-APR-2012 10:58:57.211651
End time:    08-APR-2012 11:07:24.057091
Bands:       ['radiance_1', 'radiance_2', 'radiance_3', 'radiance_4', 'radiance_5', 'radiance_6', 'radiance_7', 'radiance_8', 'radiance_9', 'radiance_10', 'radiance_11', 'radiance_12', 'radiance_13', 'radiance_14', 'radiance_15', 'l1_flags', 'detector_index']
Writing...
processing line  0  of  11521

/opt/anaconda/anaconda3/envs/python34/lib/python3.4/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda/anaconda3/envs/python34/lib/python3.4/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in less
/opt/anaconda/anaconda3/envs/python34/lib/python3.4/site-packages/ipykernel/__main__.py:55: RuntimeWarning: invalid value encountered in greater



processing line  1  of  11521
processing line  2  of  11521
processing line  3  of  11521
processing line  4  of  11521
processing line  5  of  11521
processing line  6  of  11521
processing line  7  of  11521
processing line  8  of  11521
processing line  9  of  11521
processing line  10  of  11521
processing line  11  of  11521
processing line  12  of  11521
processing line  13  of  11521
processing line  14  of  11521
processing line  15  of  11521
processing line  16  of  11521
processing line  17  of  11521
processing line  18  of  11521
processing line  19  of  11521
processing line  20  of  11521
processing line  21  of  11521
processing line  22  of  11521
processing line  23  of  11521
processing line  24  of  11521
processing line  25  of  11521
processing line  26  of  11521
processing line  27  of  11521
processing line  28  of  11521
processing line  29  of  11521
processing line  30  of  11521
processing line  31  of  11521
processing line  32  of  11521
processing line 

processing line  326  of  11521
processing line  327  of  11521
processing line  328  of  11521
processing line  329  of  11521
processing line  330  of  11521
processing line  331  of  11521
processing line  332  of  11521
processing line  333  of  11521
processing line  334  of  11521
processing line  335  of  11521
processing line  336  of  11521
processing line  337  of  11521
processing line  338  of  11521
processing line  339  of  11521
processing line  340  of  11521
processing line  341  of  11521
processing line  342  of  11521
processing line  343  of  11521
processing line  344  of  11521
processing line  345  of  11521
processing line  346  of  11521
processing line  347  of  11521
processing line  348  of  11521
processing line  349  of  11521
processing line  350  of  11521
processing line  351  of  11521
processing line  352  of  11521
processing line  353  of  11521
processing line  354  of  11521
processing line  355  of  11521
processing line  356  of  11521
processi

processing line  592  of  11521
processing line  593  of  11521
processing line  594  of  11521
processing line  595  of  11521
processing line  596  of  11521
processing line  597  of  11521
processing line  598  of  11521
processing line  599  of  11521
processing line  600  of  11521
processing line  601  of  11521
processing line  602  of  11521
processing line  603  of  11521
processing line  604  of  11521
processing line  605  of  11521
processing line  606  of  11521
processing line  607  of  11521
processing line  608  of  11521
processing line  609  of  11521
processing line  610  of  11521
processing line  611  of  11521
processing line  612  of  11521
processing line  613  of  11521
processing line  614  of  11521
processing line  615  of  11521
processing line  616  of  11521
processing line  617  of  11521
processing line  618  of  11521
processing line  619  of  11521
processing line  620  of  11521
processing line  621  of  11521
processing line  622  of  11521
processi

...

processing line  9850  of  11521
processing line  9851  of  11521
processing line  9852  of  11521
processing line  9853  of  11521
processing line  9854  of  11521
processing line  9855  of  11521
processing line  9856  of  11521
processing line  9857  of  11521
processing line  9858  of  11521
processing line  9859  of  11521
processing line  9860  of  11521
processing line  9861  of  11521
processing line  9862  of  11521
processing line  9863  of  11521
processing line  9864  of  11521
processing line  9865  of  11521
processing line  9866  of  11521
processing line  9867  of  11521
processing line  9868  of  11521
processing line  9869  of  11521
processing line  9870  of  11521
processing line  9871  of  11521
processing line  9872  of  11521
processing line  9873  of  11521
processing line  9874  of  11521
processing line  9875  of  11521
processing line  9876  of  11521
processing line  9877  of  11521
processing line  9878  of  11521
processing line  9879  of  11521
processing

processing line  10206  of  11521
processing line  10207  of  11521
processing line  10208  of  11521
processing line  10209  of  11521
processing line  10210  of  11521
processing line  10211  of  11521
processing line  10212  of  11521
processing line  10213  of  11521
processing line  10214  of  11521
processing line  10215  of  11521
processing line  10216  of  11521
processing line  10217  of  11521
processing line  10218  of  11521
processing line  10219  of  11521
processing line  10220  of  11521
processing line  10221  of  11521
processing line  10222  of  11521
processing line  10223  of  11521
processing line  10224  of  11521
processing line  10225  of  11521
processing line  10226  of  11521
processing line  10227  of  11521
processing line  10228  of  11521
processing line  10229  of  11521
processing line  10230  of  11521
processing line  10231  of  11521
processing line  10232  of  11521
processing line  10233  of  11521
processing line  10234  of  11521
processing lin

processing line  10551  of  11521
processing line  10552  of  11521
processing line  10553  of  11521
processing line  10554  of  11521
processing line  10555  of  11521
processing line  10556  of  11521
processing line  10557  of  11521
processing line  10558  of  11521
processing line  10559  of  11521
processing line  10560  of  11521
processing line  10561  of  11521
processing line  10562  of  11521
processing line  10563  of  11521
processing line  10564  of  11521
processing line  10565  of  11521
processing line  10566  of  11521
processing line  10567  of  11521
processing line  10568  of  11521
processing line  10569  of  11521
processing line  10570  of  11521
processing line  10571  of  11521
processing line  10572  of  11521
processing line  10573  of  11521
processing line  10574  of  11521
processing line  10575  of  11521
processing line  10576  of  11521
processing line  10577  of  11521
processing line  10578  of  11521
processing line  10579  of  11521
processing lin

processing line  10901  of  11521
processing line  10902  of  11521
processing line  10903  of  11521
processing line  10904  of  11521
processing line  10905  of  11521
processing line  10906  of  11521
processing line  10907  of  11521
processing line  10908  of  11521
processing line  10909  of  11521
processing line  10910  of  11521
processing line  10911  of  11521
processing line  10912  of  11521
processing line  10913  of  11521
processing line  10914  of  11521
processing line  10915  of  11521
processing line  10916  of  11521
processing line  10917  of  11521
processing line  10918  of  11521
processing line  10919  of  11521
processing line  10920  of  11521
processing line  10921  of  11521
processing line  10922  of  11521
processing line  10923  of  11521
processing line  10924  of  11521
processing line  10925  of  11521
processing line  10926  of  11521
processing line  10927  of  11521
processing line  10928  of  11521
processing line  10929  of  11521
processing lin

processing line  11238  of  11521
processing line  11239  of  11521
processing line  11240  of  11521
processing line  11241  of  11521
processing line  11242  of  11521
processing line  11243  of  11521
processing line  11244  of  11521
processing line  11245  of  11521
processing line  11246  of  11521
processing line  11247  of  11521
processing line  11248  of  11521
processing line  11249  of  11521
processing line  11250  of  11521
processing line  11251  of  11521
processing line  11252  of  11521
processing line  11253  of  11521
processing line  11254  of  11521
processing line  11255  of  11521
processing line  11256  of  11521
processing line  11257  of  11521
processing line  11258  of  11521
processing line  11259  of  11521
processing line  11260  of  11521
processing line  11261  of  11521
processing line  11262  of  11521
processing line  11263  of  11521
processing line  11264  of  11521
processing line  11265  of  11521
processing line  11266  of  11521
processing lin